# CNN For Identifying Potholes

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from PIL import Image
from annotation_parser import AnnotationParser
from features_old import extract_features
from feature_extractor import FeatureExtractor
from preprocessor import Preprocessor

### 1. Preprocessing

You mentioned you have most of the code for this section. I'll provide a brief placeholder for context.


In [ ]:
# Usage
parser = AnnotationParser()
preprocessor = Preprocessor(parser)
preprocessor.load_annotations('df1_annotations.json', 'ds2_annotations.json', 'ds3_trn.json', 'ds3_tst.json', 'ds4_trn.json', 'ds4_tst.json')
preprocessor.split_dataset('df1_splits.json')
all_data = preprocessor.preprocess()
print(all_data.head())


In [ ]:
# Load and preprocess data
def load_data(json_file, img_dir):
    df = pd.read_json(json_file, lines=True)
    images = []
    masks = []
    for _, row in df.iterrows():
        img_path = os.path.join(img_dir, row['image_path'])
        img = Image.open(img_path).resize((128, 128))  # Resize for simplicity
        img_array = np.array(img) / 255.0  # Normalize
        images.append(img_array)
        
        mask = np.zeros((img.height, img.width))
        for box in row['boxes']:
            x, y, w, h = box['x'], box['y'], box['width'], box['height']
            mask[y:y+h, x:x+w] = 1
        masks.append(mask)
    
    return np.array(images), np.array(masks)


In [ ]:
# Load data
X_train, y_train = load_data('./data/processed/json/ds3_trn.json', './data/processed/')
X_test, y_test = load_data('./data/processed/json/ds3_tst.json', './data/processed/')


### 2. Features

This section typically involves feature extraction or engineering.

For CNNs, the raw pixel values of the images are used as features.

However, if any additional feature engineering is needed, it would go here.


In [ ]:
FeatureExtractor().extract_features()


### 3. Model Design

Here, we'll define the architecture of the CNN model.



In [ ]:
# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

### 4. Model Build
In this section, we'll compile the model and train it using the training data.


In [ ]:
def build_model(model, X_train, y_train, X_val, y_val):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
    return model, history

### 5. Validation

Here, we'll evaluate the model's performance on the validation or test set.


In [ ]:
def validate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy


### Usage




In [ ]:
# Preprocessing
X_train, y_train, X_val, y_val, X_test, y_test = Preprocessor()

# Feature Extraction
extract_features()

# Model Design
model = design_model()

# Model Build
model, history = build_model(model, X_train, y_train, X_val, y_val)

# Validation
validate_model(model, X_test, y_test)

# Visualization
from visualize import NeuralNetworkVisualizer as nnv

nnv.visualize_activation_maps(model, history)
nnv.calculate_auc()
nnv.plot_roc_curve()

